In [48]:
import os
import numpy as np
import soundfile
import librosa
import h5py
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans

import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import seaborn as sns
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [49]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.net = models.resnet50(pretrained=True)
 
    def forward(self, input):
        output = self.net.conv1(input)
        output = self.net.bn1(output)
        output = self.net.relu(output)
        output = self.net.maxpool(output)
        output = self.net.layer1(output)
        output = self.net.layer2(output)
        output = self.net.layer3(output)
        output = self.net.layer4(output)
        output = self.net.avgpool(output)
        return output

def get_model():
    model = net()
    model = model.eval()
    model.cuda()
    return model

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

def extract_features(model,fea):
    model.eval()
    
    new_img = Image.fromarray(fea.astype(np.float32))
    new_img = new_img.resize((224,224))
    fea = np.array(new_img.getdata(),dtype=np.float32).reshape(224,224)
    
    fea = np.resize(fea,(224,224,3))
    tensor = normalize(to_tensor(fea))
    tensor = tensor.cuda()
    res = model(Variable(tensor).unsqueeze(0))
    res = res.data.squeeze()
    return res

In [50]:
def train(name):
    print(name)
    df = pd.DataFrame(pd.read_excel(name+'.xlsx'))
    
    audioname = []
    num = 0
    audioset = {}
    n = df.shape[0]
    for i in range(n):
        if i is not 0:
            if df.iloc[i,2] is not df.iloc[i-1,2]:
                num += 1
        audioname.append(num)
        audioset[str(df.iloc[i,2])] = num
        
    audiofeature = np.load(name+'.npy',allow_pickle=True)
    features = []
    model = get_model()
    for fea in audiofeature:
        res = extract_features(model,fea)
        features.append(res.tolist())
    print(len(features))
    
    X = np.array(features)
    y = np.array(audioname)
    x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=99)
    clf = SVC(C=20,kernel='rbf',gamma='auto',decision_function_shape='ovr')
    clf.fit(x_train, y_train)
    print(clf.score(x_test,y_test))

In [51]:
train('吹奏类')

吹奏类
2092
0.8687350835322196


In [39]:
train('弹拨类')
train('拉弦类')
train('敲击类')

弹拨类
1240
0.6653225806451613
拉弦类
1028
0.7524271844660194
敲击类
620
0.8064516129032258
